In [1]:
import imageio

import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

import PIL.Image
from io import BytesIO
from IPython.display import clear_output, Image, display

%matplotlib inline

In [2]:
def animate(mass, position):
    image = np.zeros((1024, 1024))
    position = np.round(position).astype(np.int64)
    image[np.maximum(position[0] - mass, 0) : np.minimum(position[0] + mass, 1024), 
          np.maximum(position[1] - mass, 0) : np.minimum(position[1] + mass, 1024)] = 255
    image = np.uint8(np.clip(image, 0, 255))
    
#     assert(np.sum(image==255) < (2*mass+1)**2, 'sfsdjfldkf')
#     file = BytesIO()
#     PIL.Image.fromarray(image).save(file, 'jpeg')
#     clear_output(wait = True)
#     display(Image(data=file.getvalue()))
    return image

In [3]:
DIMENSIONS = 2

In [4]:
f_initial = tf.placeholder(shape=[DIMENSIONS], dtype=tf.float64)
v_initial = tf.placeholder(shape=[DIMENSIONS], dtype=tf.float64)
x_initial = tf.placeholder(shape=[DIMENSIONS], dtype=tf.float64)

f = tf.Variable(f_initial, dtype=tf.float64)

m = tf.placeholder(shape=[], dtype=tf.float64)

a = tf.Variable(np.zeros((DIMENSIONS)), dtype=tf.float64)
v = tf.Variable(v_initial, dtype=tf.float64)
x = tf.Variable(x_initial, dtype=tf.float64)

t = tf.Variable(0.0, dtype=tf.float64)

t_h = tf.placeholder(shape=[], dtype=tf.float64)

In [5]:
f_ = f * tf.cast(tf.pow(-1, tf.cast(tf.logical_or(x < m, x > 1024 - m), dtype=tf.int32)), dtype=tf.float64)
a_ = f_ / m
v_ = v * tf.cast(tf.pow(-1, tf.cast(tf.logical_or(x < m, x > 1024 - m), dtype=tf.int32)), dtype=tf.float64)
v_ = v_ + (a_ * t) * t_h
x_ = x + (v_ * t) * t_h
t_ = t + 1
step = tf.group(f.assign(f_), 
                a.assign(a_), 
                v.assign(v_), 
                x.assign(x_), 
                t.assign(t_))

In [6]:
force_0 = np.array([0.0, 0.0], dtype=np.float64)
velocity_0 = np.array([1, 1.01], dtype=np.float64)
position_0 = np.array([512, 512])

mass = 10
time_horizon = 0.1
number_horizon = int(100 / time_horizon)

gif_frames = []

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer(), 
             feed_dict={f_initial:force_0, 
                        v_initial:velocity_0, 
                        x_initial:position_0})
    for i in range(number_horizon):
        sess.run(step, feed_dict={m:mass, t_h:time_horizon})
        x1, x2, x3, x4 = sess.run([f, a, v, x])
        gif_frames.append(animate(mass, x4))
        
imageio.mimsave('movie.gif', gif_frames)